In [1]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# postsDf = pd.read_csv("information_for_mongo.csv")

In [2]:
#time conversion and edit formats
from datetime import datetime
import time

postsDf.reset_index(drop=True)
postsDf = postsDf.fillna(value="None")
postsDf.drop(postsDf.columns[postsDf.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = {}
      row_dict = eval(row["event_details"])
      try:
        conStart = datetime.fromisoformat(row_dict["start_time"])
        unixStart = time.mktime(conStart.timetuple())
        try:
          conEnd = datetime.fromisoformat(row_dict["end_time"])
          unixEnd = time.mktime(conEnd.timetuple())
        except:
          conEnd = None
        row_dict["start_time"]=unixStart
        row_dict["end_time"]=unixEnd
      except:
        postsDf.at[index, "is_event"] = False
        err = f'Start failed {row_dict["start_time"]}'
        print(index, err)
      postsDf.at[index, "event_details"] = row_dict
      print(unixStart,unixEnd)

1725827400.0 1725836400.0
4 Start failed 
1725827400.0 1725836400.0
1725926400.0 1725930000.0
6 Start failed 
1725926400.0 1725930000.0


In [ ]:
# #resetting db if nexessary
# load_dotenv()
# uri = os.getenv('DATABASE_URI')
# client = MongoClient(uri, server_api=ServerApi('1'))
# db = client['Instagram']
# collection = db["Events"]

# collection.delete_many({})

DeleteResult({'n': 31, 'electionId': ObjectId('7fffffff0000000000000daa'), 'opTime': {'ts': Timestamp(1730605968, 94), 't': 3498}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1730605968, 94), 'signature': {'hash': b'\x15?\xe0\x91\xca\xcdF\x1b\xaa\x82\xc45"\xd0\xcf\xfb\xa0#d\x08', 'keyId': 7394494294142222396}}, 'operationTime': Timestamp(1730605968, 94)}, acknowledged=True)

In [ ]:
from typing import List
from together import Together
import json
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'BAAI/bge-large-en-v1.5' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = {}
      for column in postsDf.columns.tolist():
        if column in ['account','date','caption','accessibility_caption','url','likes','display_photo']:
          row_dict[column] = row[column]
      row_dict["embedded"] = generate_embedding([row["processed_json"]], embedding_model_string)  
      row_dict["event_details"] = row["event_details"]
      result = collection.insert_one(row_dict)
      print(f"Inserted document ID: {result.inserted_id}")
      time.sleep(1)



        

NameError: name 'postsDf' is not defined

In [6]:

def return_events(prompt):
    query = prompt
    query_emb = generate_embedding([query], embedding_model_string)[0]

    results = collection.aggregate([
    {
        "$vectorSearch": {
        "queryVector": query_emb,
        "path": "embedded",
        "numCandidates": 100, # this should be 10-20x the limit
        "limit": 2, # the number of documents to return in the results
        "index": "vector_index", # the index name you used in Step 4.
        }
    }
    ])
    response = ""
    for i in results:
        response += (i['caption']+ "\n"+ i['url'] + '\n\n')
    return response

prompt = "Cheese"

print(return_events(prompt))


Gouda vibes only this Tuesday, fellow cheese enjoyers:Cranberry White Stilton is back, along with 4 others! Feeling blue? No worries, there is Limoncello Blue to raise your spirits on this grey day!  We also have Guinness Cheddar, Savello di Roma - a semi-hard sheep’s milk cheese, and Reblochon Savoie (the soft cheese that won last week!)! ••If any of these sound “grate,” come over to RCH 103 at 6:30pm today to claim your share of this cheese haul!🧀🧀🧀🧀🧀🪤
C4JclfYgzHk

Greetings cheese enjoyers,We've got another interesting selection of cheeses this week with the cranberry stilton, last week's winner, making a return appearance.We've also got TWO lactose-free cheeses this week with the frère Jacques and jalapeño gooda. That's right, not one, but TWO! Share the news with your friends!As always, come by RCH 103 around 6:30pm today to claim your share of this cheese haul!🧀🧀🧀🧀🧀🪤
C3TSnKCAvRA


